# ICASSP Experiments

Datasets : [MMUISD]

**Data Variations**
* Features = [Accelerometer Only, Accelerometer + Gyroscope]
* Sequence Length = [1, 2, 3, 4]
* Overlap = [no overlap, 0.25, 0.5, 0.75]
* Train-Validation-Test split = [0.8, 0.1, 0.1]

**Model Variations**
* Dropout = [Yes, No]
* BN = [Yes, No]
* Weight Decay = [Yes, No]
* Weight Decay + BN
* Weight Decay + Dropout

**Report:**
* Mean EER
* Std EER
* Min EER
* Max EER

**Visualization**
* Distance Vectors of Best and Worst combinations of all datasets stored in 200 ppt.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!cp -r "/content/drive/My Drive/icmla2020/src" /content/
!mv src/datasets.py datasets.py
!mv src/models.py models.py
!mv src/analyze.py analyze.py
!ls

analyze.py  datasets.py  drive	models.py  sample_data	src


In [1]:
data_path = '../data/ejust/'
results_path = '../results/'

In [2]:
import datasets, models, analyze
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# import nevergrad as ng
np.set_printoptions(precision=4)

In [3]:
def prepare_data(train_sps, val_sps, test_sps, max_cls, split, data, labels, indices_list, run_n):
    train_split = int(split[0] * max_cls)
    val_split = int(split[2] * max_cls)

    rand_indices = np.random.choice(range(max_cls), size=max_cls, replace=False)
    indices_list.append(rand_indices)
    print('Run #%d:'%run_n)
    
    all_data_rand, all_labels_rand = [data[x] for x in rand_indices], [labels[x] for x in rand_indices]
    b_data, b_labels = datasets.generate_batch(train_sps, all_data_rand[:train_split], all_labels_rand[:train_split])
    
    val_data, val_labels = datasets.generate_batch(val_sps, all_data_rand[train_split:train_split+val_split], all_labels_rand[train_split:train_split+val_split])
    ridx = np.random.choice(range(b_data[0].shape[0]), size=b_data[0].shape[0], replace=False)
    
    b_data_test, b_labels_test = datasets.generate_batch(test_sps, all_data_rand[train_split+val_split:], all_labels_rand[train_split+val_split:])

    l_input = b_data[0][ridx]
    r_input = b_data[1][ridx]
    b_labels = b_labels[ridx]

    # print(l_input[0].shape)

    l_input_val = val_data[0]
    r_input_val = val_data[1]

    l_input_test = b_data_test[0]
    r_input_test = b_data_test[1]
    return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test

In [4]:
def load_dataset(segLen, overlap, acc_only):
    data, labels = datasets.load_segment_EJUST(data_path, 'D5-LC', [0, 20], segment_time=2)
    return data, labels

In [5]:
def build_model1(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(64, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [6]:
def build_model2(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(32, 3, strides=2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape))
        model.add(Conv1D(32, 3, strides=2, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(MaxPooling1D(pool_size=2))

    if reg:
        model.add(Conv1D(64, 5, strides=2, activation='tanh', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(128, 5, strides=3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2, activation='tanh'))
        model.add(Conv1D(128, 5, strides=3, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(GlobalMaxPooling1D())
    return model

In [7]:
def build_model3(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    model.add(Conv1D(64, 5, strides=3, padding='valid', activation='relu', input_shape=input_shape))
    if bn: model.add(BatchNormalization())
    model.add(Conv1D(128, 3, strides=2, padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, 2, strides=1, padding='valid', activation='tanh'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(GlobalMaxPooling1D())
    return model

In [8]:
def build_model4(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(128, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(128, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [9]:
def build_model5(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(256, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(256, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [10]:
acc_only_list    = [False, True]
segLen_list      = [1, 2]
overlap_list     = [0, 0.25, 0.5, 0.75]
# models_list      = [build_model1, build_model2, build_model3]
models_list      = [build_model1, build_model4, build_model5]
bn_list          = [True, False]
reg_list         = [True, False]

# Configurations
train_sps       = 500
val_sps         = 200
test_sps        = 200
max_cls         = 20
split           = [0.5, 0.25, 0.25]
output_dropout  = 0.1
runs            = 30
np.set_printoptions(precision=4)

In [ ]:
data, labels = load_dataset(2, 0, acc_only=False)

# Results of this configuration/experiment
EERs = np.array([np.zeros(runs) for _ in range(3*2*2)])

for run_n in range(runs):
    l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test = prepare_data(train_sps,
                                                                                                                                val_sps,
                                                                                                                                test_sps,
                                                                                                                                max_cls,
                                                                                                                                split,
                                                                                                                                data,
                                                                                                                                labels,
                                                                                                                                [],
                                                                                                                                run_n)
    adam = Adam(learning_rate=0.001)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
    for bn in bn_list:
        for reg in reg_list:
            for model_index, build_cnn in enumerate(models_list):
                cnn = build_cnn(l_input[0].shape, bn, reg)
                siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
                siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
                if run_n == 0: 
                    cnn.summary()
                hist = siamese.fit([l_input, r_input], 
                                    b_labels, 
                                    shuffle=True,
                                    batch_size=64,
                                    epochs=100,
                                    callbacks=[earlystop],
                                    validation_data=([l_input_val, r_input_val], val_labels),
                                    verbose=2
                                    )
                FRR, FAR, EER, EER_th = analyze.ROC(siamese, [l_input_test, r_input_test], b_labels_test)
                print('MODEL(%d) BN(%d) REG(%d) EER: %g', (model_index, bn, reg, EER))
                EERs[4 * model_index + 2*bn + reg , run_n] = EER
                np.save(results_path + 'ejust_filters_eers3.npy', EERs)


Run #0:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 98, 16)            304       
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 16)            64        
_________________________________________________________________
activation (Activation)      (None, 98, 16)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 64)            5184      
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 64)            256       
_________________________________________________________________
activation_1 (Activation)    (None, 23, 64)     

Epoch 13/100
10000/10000 - 2s - loss: 0.0319 - accuracy: 0.9996 - val_loss: 0.0532 - val_accuracy: 0.9945
Epoch 14/100
10000/10000 - 2s - loss: 0.0324 - accuracy: 0.9986 - val_loss: 0.0636 - val_accuracy: 0.9915
Epoch 15/100
10000/10000 - 2s - loss: 0.0681 - accuracy: 0.9938 - val_loss: 0.0845 - val_accuracy: 0.9890
MODEL(%d) BN(%d) REG(%d) EER: %g (2, True, True, 0.199)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 98, 16)            304       
_________________________________________________________________
batch_normalization_6 (Batch (None, 98, 16)            64        
_________________________________________________________________
activation_6 (Activation)    (None, 98, 16)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 49, 16)            0         
__________

Epoch 9/100
10000/10000 - 2s - loss: 0.0067 - accuracy: 0.9989 - val_loss: 0.0337 - val_accuracy: 0.9885
Epoch 10/100
10000/10000 - 2s - loss: 0.0072 - accuracy: 0.9992 - val_loss: 0.0451 - val_accuracy: 0.9820
Epoch 11/100
10000/10000 - 2s - loss: 0.0051 - accuracy: 0.9994 - val_loss: 0.0495 - val_accuracy: 0.9835
Epoch 12/100
10000/10000 - 2s - loss: 0.0128 - accuracy: 0.9976 - val_loss: 0.0526 - val_accuracy: 0.9800
Epoch 13/100
10000/10000 - 2s - loss: 0.0030 - accuracy: 0.9996 - val_loss: 0.0591 - val_accuracy: 0.9785
MODEL(%d) BN(%d) REG(%d) EER: %g (2, True, False, 0.14)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 98, 16)            304       
_________________________________________________________________
activation_12 (Activation)   (None, 98, 16)            0         
_______________________________________________________________

Epoch 11/100
10000/10000 - 1s - loss: 0.0710 - accuracy: 0.9960 - val_loss: 0.1149 - val_accuracy: 0.9840
Epoch 12/100
10000/10000 - 1s - loss: 0.0558 - accuracy: 0.9991 - val_loss: 0.1074 - val_accuracy: 0.9840
Epoch 13/100
10000/10000 - 1s - loss: 0.0543 - accuracy: 0.9990 - val_loss: 0.1088 - val_accuracy: 0.9845
Epoch 14/100
10000/10000 - 1s - loss: 0.0735 - accuracy: 0.9944 - val_loss: 0.1393 - val_accuracy: 0.9810
MODEL(%d) BN(%d) REG(%d) EER: %g (2, False, True, 0.13)
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 98, 16)            304       
_________________________________________________________________
activation_18 (Activation)   (None, 98, 16)            0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 49, 16)            0         
____________________________________

Train on 10000 samples, validate on 2000 samples
Epoch 1/100
10000/10000 - 2s - loss: 0.8096 - accuracy: 0.9206 - val_loss: 0.2078 - val_accuracy: 0.9460
Epoch 2/100
10000/10000 - 1s - loss: 0.1109 - accuracy: 0.9792 - val_loss: 0.1308 - val_accuracy: 0.9640
Epoch 3/100
10000/10000 - 1s - loss: 0.0605 - accuracy: 0.9883 - val_loss: 0.1431 - val_accuracy: 0.9600
Epoch 4/100
10000/10000 - 1s - loss: 0.0506 - accuracy: 0.9904 - val_loss: 0.1005 - val_accuracy: 0.9730
Epoch 5/100
10000/10000 - 1s - loss: 0.0307 - accuracy: 0.9947 - val_loss: 0.0841 - val_accuracy: 0.9735
Epoch 6/100
10000/10000 - 1s - loss: 0.0136 - accuracy: 0.9958 - val_loss: 0.0633 - val_accuracy: 0.9790
Epoch 7/100
10000/10000 - 1s - loss: 0.0134 - accuracy: 0.9971 - val_loss: 0.0780 - val_accuracy: 0.9755
Epoch 8/100
10000/10000 - 1s - loss: 0.0139 - accuracy: 0.9970 - val_loss: 0.0617 - val_accuracy: 0.9830
Epoch 9/100
10000/10000 - 1s - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.0743 - val_accuracy: 0.9810
Epoch 

...

Epoch 10/100
10000/10000 - 1s - loss: 0.0342 - accuracy: 0.9924 - val_loss: 0.4025 - val_accuracy: 0.8825
Epoch 11/100
10000/10000 - 1s - loss: 0.0353 - accuracy: 0.9921 - val_loss: 0.5204 - val_accuracy: 0.8570
Epoch 12/100
10000/10000 - 1s - loss: 0.0320 - accuracy: 0.9943 - val_loss: 0.5544 - val_accuracy: 0.8440
Epoch 13/100
10000/10000 - 1s - loss: 0.0335 - accuracy: 0.9937 - val_loss: 0.4320 - val_accuracy: 0.8690
Epoch 14/100
10000/10000 - 1s - loss: 0.0323 - accuracy: 0.9918 - val_loss: 0.3938 - val_accuracy: 0.8885
Epoch 15/100
10000/10000 - 1s - loss: 0.0270 - accuracy: 0.9943 - val_loss: 0.4373 - val_accuracy: 0.8705
Epoch 16/100
10000/10000 - 1s - loss: 0.0335 - accuracy: 0.9931 - val_loss: 0.4527 - val_accuracy: 0.8595
Epoch 17/100
10000/10000 - 1s - loss: 0.0270 - accuracy: 0.9948 - val_loss: 0.3737 - val_accuracy: 0.8830
Epoch 18/100
10000/10000 - 1s - loss: 0.0229 - accuracy: 0.9963 - val_loss: 0.3801 - val_accuracy: 0.8885
Epoch 19/100
10000/10000 - 1s - loss: 0.0205 -

10000/10000 - 2s - loss: 0.5643 - accuracy: 0.9097 - val_loss: 0.7373 - val_accuracy: 0.8610
Epoch 2/100
10000/10000 - 1s - loss: 0.0806 - accuracy: 0.9796 - val_loss: 0.6789 - val_accuracy: 0.8800
Epoch 3/100
10000/10000 - 1s - loss: 0.0591 - accuracy: 0.9860 - val_loss: 0.6479 - val_accuracy: 0.8695
Epoch 4/100
10000/10000 - 1s - loss: 0.0254 - accuracy: 0.9931 - val_loss: 0.3764 - val_accuracy: 0.9085
Epoch 5/100
10000/10000 - 1s - loss: 0.0146 - accuracy: 0.9956 - val_loss: 0.4558 - val_accuracy: 0.8960
Epoch 6/100
10000/10000 - 1s - loss: 0.0161 - accuracy: 0.9957 - val_loss: 0.5116 - val_accuracy: 0.8820
Epoch 7/100
10000/10000 - 1s - loss: 0.0097 - accuracy: 0.9968 - val_loss: 0.5086 - val_accuracy: 0.8890
Epoch 8/100
10000/10000 - 1s - loss: 0.0104 - accuracy: 0.9973 - val_loss: 0.4322 - val_accuracy: 0.9035
Epoch 9/100
10000/10000 - 1s - loss: 0.0138 - accuracy: 0.9960 - val_loss: 0.4366 - val_accuracy: 0.9035
MODEL(%d) BN(%d) REG(%d) EER: %g (1, False, False, 0.203)
Train on 

10000/10000 - 3s - loss: 0.1309 - accuracy: 0.9641 - val_loss: 0.4233 - val_accuracy: 0.8515
Epoch 2/100
10000/10000 - 2s - loss: 0.0333 - accuracy: 0.9949 - val_loss: 0.3607 - val_accuracy: 0.8710
Epoch 3/100
10000/10000 - 2s - loss: 0.0183 - accuracy: 0.9975 - val_loss: 0.2975 - val_accuracy: 0.8925
Epoch 4/100
10000/10000 - 2s - loss: 0.0136 - accuracy: 0.9975 - val_loss: 0.3507 - val_accuracy: 0.8870
Epoch 5/100
10000/10000 - 2s - loss: 0.0096 - accuracy: 0.9991 - val_loss: 0.4053 - val_accuracy: 0.8720
Epoch 6/100
10000/10000 - 2s - loss: 0.0075 - accuracy: 0.9988 - val_loss: 0.5579 - val_accuracy: 0.8510
Epoch 7/100
10000/10000 - 2s - loss: 0.0063 - accuracy: 0.9991 - val_loss: 0.4125 - val_accuracy: 0.8780
Epoch 8/100
10000/10000 - 2s - loss: 0.0042 - accuracy: 0.9994 - val_loss: 0.4882 - val_accuracy: 0.8675
MODEL(%d) BN(%d) REG(%d) EER: %g (2, True, False, 0.095)
Train on 10000 samples, validate on 2000 samples
Epoch 1/100
10000/10000 - 2s - loss: 0.5491 - accuracy: 0.9031 - v

Epoch 5/100
10000/10000 - 1s - loss: 0.0368 - accuracy: 0.9913 - val_loss: 0.3443 - val_accuracy: 0.9275
Epoch 6/100
10000/10000 - 1s - loss: 0.0268 - accuracy: 0.9945 - val_loss: 0.3267 - val_accuracy: 0.9280
Epoch 7/100
10000/10000 - 1s - loss: 0.0197 - accuracy: 0.9955 - val_loss: 0.3172 - val_accuracy: 0.9305
Epoch 8/100
10000/10000 - 1s - loss: 0.0212 - accuracy: 0.9956 - val_loss: 0.4452 - val_accuracy: 0.9020
Epoch 9/100
10000/10000 - 1s - loss: 0.0292 - accuracy: 0.9928 - val_loss: 0.3603 - val_accuracy: 0.9200
Epoch 10/100
10000/10000 - 1s - loss: 0.0197 - accuracy: 0.9956 - val_loss: 0.3504 - val_accuracy: 0.9330
Epoch 11/100
10000/10000 - 1s - loss: 0.0135 - accuracy: 0.9969 - val_loss: 0.2643 - val_accuracy: 0.9455
Epoch 12/100
10000/10000 - 1s - loss: 0.0099 - accuracy: 0.9973 - val_loss: 0.3486 - val_accuracy: 0.9200
Epoch 13/100
10000/10000 - 1s - loss: 0.0128 - accuracy: 0.9973 - val_loss: 0.2452 - val_accuracy: 0.9420
Epoch 14/100
10000/10000 - 1s - loss: 0.0059 - accu

Epoch 3/100
10000/10000 - 1s - loss: 0.0671 - accuracy: 0.9853 - val_loss: 0.2213 - val_accuracy: 0.9120
Epoch 4/100
10000/10000 - 1s - loss: 0.0538 - accuracy: 0.9884 - val_loss: 0.4512 - val_accuracy: 0.8385
Epoch 5/100
10000/10000 - 1s - loss: 0.0424 - accuracy: 0.9911 - val_loss: 1.1132 - val_accuracy: 0.7650
Epoch 6/100
10000/10000 - 1s - loss: 0.0373 - accuracy: 0.9927 - val_loss: 0.9035 - val_accuracy: 0.7815
Epoch 7/100
10000/10000 - 1s - loss: 0.0311 - accuracy: 0.9937 - val_loss: 0.3065 - val_accuracy: 0.8915
Epoch 8/100
10000/10000 - 1s - loss: 0.0281 - accuracy: 0.9946 - val_loss: 0.5160 - val_accuracy: 0.8375
MODEL(%d) BN(%d) REG(%d) EER: %g (0, True, False, 0.119)
Train on 10000 samples, validate on 2000 samples
Epoch 1/100
10000/10000 - 3s - loss: 0.1582 - accuracy: 0.9506 - val_loss: 0.2424 - val_accuracy: 0.9245
Epoch 2/100
10000/10000 - 1s - loss: 0.0463 - accuracy: 0.9896 - val_loss: 0.3020 - val_accuracy: 0.9195
Epoch 3/100
10000/10000 - 1s - loss: 0.0311 - accuracy

In [ ]:
EERs = np.load(results_path + 'ejust_filters_eers3.npy')
EERs = EERs[:8]
print(EERs.shape)

In [ ]:
# EERs[EERs == 0] = 100
means = np.mean(EERs, axis=1) * 100
print(means)

stds = np.std(EERs, axis=1) * 100
print(stds)

mins = np.min(EERs, axis=1) * 100
print(mins)

mins = np.max(EERs, axis=1) * 100
print(mins)

In [ ]:
import pandas as pd
E = pd.DataFrame(EERs)
print(E.iloc[:,:15])